Practical Section (Day 1 Implementation)


In [4]:
! pip install transformers datasets torch


In [5]:
#Task 1: Load Pretrained Transformer for Classification
#First use pipeline:


from transformers import pipeline
classifier = pipeline("sentiment-analysis")
print(classifier("Transformers are powerful models."))

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f.
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading weights:   0%|          | 0/104 [00:00<?, ?it/s]

[{'label': 'POSITIVE', 'score': 0.9998114705085754}]


In [6]:
#Now loading manually:

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model_name = "distilbert-base-uncased-finetuned-sst-2-english"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

text = "Transformers are powerful models."
inputs = tokenizer(text, return_tensors="pt")

outputs = model(**inputs)
logits = outputs.logits

prediction = torch.argmax(logits, dim=1)
print(prediction)


Loading weights:   0%|          | 0/104 [00:00<?, ?it/s]

tensor([1])


In [7]:
#Task 2: Inspect Hidden States
#Enable hidden states:

from transformers import AutoTokenizer, AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    output_hidden_states=True
)

outputs = model(**inputs)
hidden_states = outputs.hidden_states

print(len(hidden_states))  # number of layers
print(hidden_states[0].shape)


Loading weights:   0%|          | 0/104 [00:00<?, ?it/s]

7
torch.Size([1, 7, 768])


In [9]:
from transformers.utils import logging
logging.set_verbosity_error()

from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

model_name = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(
    model_name,
    output_hidden_states=True
)

sentence1 = "The bank approved the loan."
sentence2 = "The river bank is beautiful."

inputs1 = tokenizer(sentence1, return_tensors="pt")
inputs2 = tokenizer(sentence2, return_tensors="pt")

outputs1 = model(**inputs1)
outputs2 = model(**inputs2)


hidden1 = outputs1.hidden_states[-1]
hidden2 = outputs2.hidden_states[-1]


tokens1 = tokenizer.tokenize(sentence1)
tokens2 = tokenizer.tokenize(sentence2)

bank_index_1 = tokens1.index("bank")
bank_index_2 = tokens2.index("bank")


bank_embedding_1 = hidden1[0, bank_index_1]
bank_embedding_2 = hidden2[0, bank_index_2]


similarity = F.cosine_similarity(
    bank_embedding_1,
    bank_embedding_2,
    dim=0
)

print("Sentence 1 tokens:", tokens1)
print("Sentence 2 tokens:", tokens2)
print("Cosine similarity between 'bank' embeddings:", similarity.item())

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

Sentence 1 tokens: ['the', 'bank', 'approved', 'the', 'loan', '.']
Sentence 2 tokens: ['the', 'river', 'bank', 'is', 'beautiful', '.']
Cosine similarity between 'bank' embeddings: 0.2699149549007416
